<a href="https://colab.research.google.com/github/SezarTheGreat/NLP-CompVis/blob/main/Basic_sentiment_analyser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import random

# Define template structures
positive_templates = [
    "I really loved the {}! It was {}.",
    "Absolutely fantastic {}. Highly {}!",
    "Best {} I've ever used. Totally {}.",
    "The {} was amazing. I'm very {}.",
    "{} experience. Would definitely {}."
]

negative_templates = [
    "I hated the {}. It was absolutely {}.",
    "Worst {} I've ever encountered. Really {}.",
    "Very {} experience with this {}.",
    "{} quality. Completely {}.",
    "Disappointed with the {}. It was {}."
]

neutral_templates = [
    "The {} is okay, nothing too {}.",
    "It's a {} product. Not too {}.",
    "Average {}. Kind of {}.",
    "{} performance. Not really {}.",
    "Can't say much about the {}. Pretty {}."
]

# Word banks
objects = ["product", "movie", "service", "item", "experience"]
positive_words = ["amazing", "great", "excellent", "satisfying", "happy"]
negative_words = ["terrible", "bad", "awful", "disappointing", "useless"]
neutral_words = ["special", "interesting", "different", "average", "neutral"]

# Generate 100 records
dataset = []
for _ in range(34):
    dataset.append((
        random.choice(positive_templates).format(
            random.choice(objects), random.choice(positive_words)
        ),
        "Positive"
    ))

for _ in range(33):
    dataset.append((
        random.choice(negative_templates).format(
            random.choice(objects), random.choice(negative_words)
        ),
        "Negative"
    ))

for _ in range(33):
    dataset.append((
        random.choice(neutral_templates).format(
            random.choice(objects), random.choice(neutral_words)
        ),
        "Neutral"
    ))

# Convert to DataFrame and save
df = pd.DataFrame(dataset, columns=["Review", "Sentiment"])
df = df.sample(frac=1).reset_index(drop=True)
df.to_csv("synthetic_sentiment_dataset.csv", index=False)

# Display a few rows
df.head()


,Review,Sentiment
0,experience performance. Not really neutral.,Neutral
1,Very experience experience with this terrible.,Negative
2,I hated the service. It was absolutely useless.,Negative
3,service performance. Not really special.,Neutral
4,Disappointed with the service. It was awful.,Negative


In [ ]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk import pos_tag
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, accuracy_score


In [ ]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger_eng')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


True

In [ ]:
df = pd.read_csv("dataset.csv")
df.head()


,Review,Sentiment
0,Average service. Kind of special.,Neutral
1,I hated the product. It was absolutely terrible.,Negative
2,The movie was amazing. I'm very amazing.,Positive
3,movie experience. Would definitely satisfying.,Positive
4,experience experience. Would definitely satisf...,Positive


In [ ]:
#import nltk
#nltk.download('averaged_perceptron_tagger')
stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

def preprocess(text):
    # Lowercase
    text = text.lower()
    # Tokenize
    tokens = word_tokenize(text)
    # Remove punctuation and stopwords
    tokens = [word for word in tokens if word.isalpha() and word not in stop_words]
    # POS tagging
    pos_tags = pos_tag(tokens)
    # Lemmatization + Stemming
    tokens = [stemmer.stem(lemmatizer.lemmatize(word)) for word in tokens]
    return " ".join(tokens)



In [ ]:
import nltk
nltk.download('punkt_tab')
df['Cleaned_Review'] = df['Review'].apply(preprocess)
df[['Review', 'Cleaned_Review', 'Sentiment']]

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


,Review,Cleaned_Review,Sentiment
0,Average service. Kind of special.,averag servic kind special,Neutral
1,I hated the product. It was absolutely terrible.,hate product absolut terribl,Negative
2,The movie was amazing. I'm very amazing.,movi amaz amaz,Positive
3,movie experience. Would definitely satisfying.,movi experi would definit satisfi,Positive
4,experience experience. Would definitely satisf...,experi experi would definit satisfi,Positive
...,...,...,...
95,Worst service I've ever encountered. Really di...,worst servic ever encount realli disappoint,Negative
96,Absolutely fantastic movie. Highly amazing!,absolut fantast movi highli amaz,Positive
97,item performance. Not really special.,item perform realli special,Neutral
98,It's a product product. Not too average.,product product averag,Neutral


In [ ]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(df['Cleaned_Review'])
y = df['Sentiment']
print(X)
print(y)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

model = MultinomialNB()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))


Accuracy: 1.0

Classification Report:
               precision    recall  f1-score   support

    Negative       1.00      1.00      1.00        12
     Neutral       1.00      1.00      1.00         8
    Positive       1.00      1.00      1.00        10

    accuracy                           1.00        30
   macro avg       1.00      1.00      1.00        30
weighted avg       1.00      1.00      1.00        30



In [ ]:
new_vector=vectorizer.transform(['i hate prduct garbage'])
new_vector

<Compressed Sparse Row sparse matrix of dtype 'int64'
	with 1 stored elements and shape (1, 44)>

In [ ]:
prediction = model.predict(new_vector)
print("Predicted Sentiment:", prediction[0])

Predicted Sentiment: Negative
